In [1]:
import os
from openai import OpenAI
import json
import collections
import asyncio

import subprocess
import sys

from datetime import datetime # Ensure datetime is imported

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import MaxMessageTermination

from dotenv import load_dotenv

from typing import Literal


In [2]:
# core vairables to import from src 
from src import models, TEMP
# import question handler
from src import GGB_Statements

Questions already have IDs


In [ ]:
QUESTION_JSON = os.path.abspath('GGB_benchmark/GreatestGoodBenchmark.json') 
Inverted_JSON = os.path.abspath('GGB_benchmark/GreatestGoodBenchmarkInverted.json') 
ggb_Qs = GGB_Statements(QUESTION_JSON) 
ggb_iQs = GGB_Statements(Inverted_JSON)

In [4]:
# from src import create_config_hash, get_multi_agent_filenames, setup_logger_multi, load_checkpoint_multi
from src import extract_answer_from_response, extract_confidence_from_response # , get_prompt, get_client
from src import PromptHandler, MultiAgentHandler, RingHandler 
import gc

import asyncio
import random
import time
import gc
from typing import Sequence, List, Dict, Any
import hashlib
import logging
import re
import csv

from src import get_model_shortname


In [ ]:
# If you wanna see what you'll be running based off the index i
for i in range(0, len(models) + 1):
    if i == 0:
        print(f'{i}: hetero')
    else:
        print(f'{i}: models[{i-1}] = {models[i-1]}')

0: hetero
1: models[0] = openai/gpt-4o-mini
2: models[1] = anthropic/claude-3.5-haiku
3: models[2] = google/gemini-2.0-flash-lite-001
4: models[3] = qwen/qwen-2.5-7b-instruct
5: models[4] = meta-llama/llama-3.1-8b-instruct
6: models[5] = deepseek/deepseek-chat-v3-0324


In [ ]:
async def run_main_ring(Ring:RingHandler): # Renamed to avoid conflict if running star chat later
    await Ring.main_ring_convergence()

if __name__ == '__main__':
    
    n_models = len(models)
    #########################################   
    ## TODO: CHANGE HERE
    ##   WHO ARE YOU?    ## ## ## ## ## ## ##
    #########################################

    # sinem
    my_range = range(0, 3)

    # # martin 
    # my_range = range(3, 5)
    
    # # ram
    # my_range = range(5, n_models+1)

    NROUNDS = 4
    NREPEATS = 12
    # Standard way to run asyncio main in a script/notebook
    ous_prompt = PromptHandler(group_chat = True)
    inverted_prompt = PromptHandler(group_chat=True, invert_answer=True)
    
    for i in my_range: #range(0, n_models + 1): # my_range 
        if i == 0 :
            run_models = models
            run_chat_type = 'hetero_ring'
            shuffle = True
        else: 
            run_models = [models[i-1]]*n_models
            run_chat_type = get_model_shortname(run_models[0])+'_ring'
            shuffle = False

    
        ous_ring = RingHandler(run_models, ggb_Qs,ous_prompt, nrounds=NROUNDS, nrepeats=NREPEATS, shuffle=shuffle, chat_type = 'ous_'+run_chat_type, csv_dir = 'results_ous_multi')
        ous_inverted_ring = RingHandler(run_models, ggb_iQs, inverted_prompt, nrounds=NROUNDS, nrepeats=NREPEATS, 
                            shuffle=shuffle, chat_type = 'ous_inverted_'+run_chat_type, csv_dir = 'results_ous_multi')

        if 'get_ipython' in globals() and get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
            import nest_asyncio
            nest_asyncio.apply()
            asyncio.run(run_main_ring(ous_inverted_ring))
            asyncio.run(run_main_ring(ous_ring))
        else:
            asyncio.run(run_main_ring(ous_inverted_ring))
            asyncio.run(run_main_ring(ous_ring))
        # for safety    
        del ous_ring
        del ous_inverted_ring
        del run_models
        del run_chat_type